Data set : https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

cleaning : https://colab.research.google.com/drive/1nwe1Ns9bJOv6zFhBXGzgNVXh9sJv52NW#scrollTo=Iw1cvXeALXDu


In [1]:
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("MyApp") \
    .master("local[1]") \
    .getOrCreate()

In [4]:
movie = spark.read.csv('movie.csv')

In [5]:
movie.count()

4809

In [6]:
new_column_names = ['movie_id','title','overview','genres','keywords','cast','crew']

In [7]:
movie = movie.toDF(*new_column_names)

In [8]:
movie.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   19995|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|
|     285|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|
|  206647|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|
|   49026|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|
|   49529|         John Carter|John Carte

In [9]:
movie.write.csv('movie_scikit.csv', header=True, mode='overwrite')

In [10]:
from pyspark.sql.functions import col
movie = movie.drop("movie_id")
movie.show(10)


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|            overview|              genres|            keywords|                cast|                crew|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|
|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|
|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|
|The Dark Knight R...|Following the dea...|['Action', 'Crime...|['dc comics', 'cr...|['Christian Bale'...|['Christopher Nol...|
|         John Carter|John Carter is a ...|['Action', 'Adven...|['based on novel'...|['Taylor Kitsch',..

In [11]:
from pyspark.sql.functions import monotonically_increasing_id
movie = movie.withColumn("movie_id", monotonically_increasing_id() + 1)


In [12]:
from pyspark.sql.functions import col
movie = movie.select("movie_id", *[col for col in movie.columns if col != "movie_id"])

In [13]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.functions import udf, col, when
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import countDistinct
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, RegexTokenizer, CountVectorizer, IDF
from pyspark.mllib.feature import IDF

from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [27]:
concat_udf = F.udf(lambda cols: " ".join([x if x is not None else "*" for x in cols]), StringType())


ConnectionRefusedError: ignored

In [15]:
movie_datacont = movie.withColumn("desc", concat_udf(F.array('genres','keywords','cast')))
movie = movie_datacont.withColumn("desc", F.regexp_replace("desc", "[/(,)]", " "))

In [16]:
from pyspark.sql.functions import regexp_replace


In [17]:
movie.show(5)

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|               title|            overview|              genres|            keywords|                cast|                crew|                desc|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       1|              Avatar|In the 22nd centu...|['Action', 'Adven...|['culture clash',...|['Sam Worthington...|   ['James Cameron']|['Action'  'Adven...|
|       2|Pirates of the Ca...|Captain Barbossa,...|['Adventure', 'Fa...|"['ocean', 'drug ...|           'traitor'|         'shipwreck'|['Adventure'  'Fa...|
|       3|             Spectre|A cryptic message...|['Action', 'Adven...|['spy', 'based on...|['Daniel Craig', ...|      ['Sam Mendes']|['Action'  'Adven...|
|       4|The Dark Knight R...|Following the dea...|

In [18]:
movie = movie_datacont.withColumn('desc', regexp_replace(col('desc'), '[^A-Za-z]+',' '))

In [19]:
movie.select('desc').show(2, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|desc                                                                                                                                                                                                                                                                                     |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe ali

In [20]:
from pyspark.sql.functions import lower
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, HashingTF, IDF


In [21]:
regexTokenizer = RegexTokenizer(inputCol="desc", outputCol="tokens", pattern='\\W')
movie = regexTokenizer.transform(movie)

In [26]:
movie.show(5)

ConnectionRefusedError: ignored

In [23]:
hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures")
featurizedData = hashingTF.transform(movie)

In [24]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)


In [25]:
feature_vectors = np.array(rescaledData.select("features").rdd.map(lambda x: x.features.toArray()).collect())

cosine_sim_matrix = cosine_similarity(feature_vectors, feature_vectors)


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 49736)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/usr/local/lib/python3.10/dist-packages/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "

ConnectionRefusedError: ignored

In [ ]:
# movie.select('tokens').show(2, False)

In [ ]:
# movie.show(5)

In [ ]:
# tfv = TfidfVectorizer(min_df = 3, max_features = None, strip_accents = 'unicode', analyzer = 'word', token_pattern =r'\w{1,}',ngram_range = (1,2),stop_words = 'english')

In [ ]:
# docs = movie.select('tokens').rdd.map(lambda x: x[0]).collect()
# docs = [str(doc).lower() for doc in docs]
# tfidf_matrix = tfv.fit_transform(docs)

# cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# docs

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType
import numpy as np

def recommendation_by_id(index):
    cosine_score = list(enumerate(cos_sim[index]))
    cosine_score = sorted(cosine_score, key=lambda x: x[1], reverse=True)
    # Top 10 recommendations based on content
    recommendations = cosine_score[1:11]  # Adjusted to retrieve top 10 recommendations
    recommend = [i[0] for i in recommendations]
    cosine_scores = [float(i[1]) for i in recommendations]
    recom = spark.createDataFrame(zip(recommend, cosine_scores), schema=["movie_id", "cosine_score"])
    recommended_movies = recom.join(movie, recom.movie_id == movie.movie_id, 'inner').select(recom.movie_id, movie.title, recom.cosine_score)
    recommended_movies.show()



In [ ]:
# Replace the index value with the desired movie index
index = 12
movie_info = movie.filter(movie['movie_id'] == index).select("movie_id", "title").first()
print("Movie ID:", movie_info.movie_id)
print("Movie Title:", movie_info.title)
print("\n")
print("Recommendation\n")
recommendation_by_id(index)


In [ ]:
def recommendation_by_title(title):
    movie_info = movie.filter(movie['title'] == title).select("movie_id", "title").first()
    if movie_info is None:
        print("Movie not found.")
        return

    index = int(movie_info.movie_id)  # Convert to integer

    cosine_score = list(enumerate(cos_sim[index]))
    cosine_score = sorted(cosine_score, key=lambda x: x[1], reverse=True)
    # Top 10 recommendations based on content
    recommendations = cosine_score[1:11]  # Adjusted to retrieve top 10 recommendations
    recommend = [i[0] for i in recommendations]
    cosine_scores = [float(i[1]) for i in recommendations]
    recom = spark.createDataFrame(zip(recommend, cosine_scores), schema=["movie_id", "cosine_score"])
    recommended_movies = recom.join(movie, recom.movie_id == movie.movie_id, 'inner').select(recom.movie_id, movie.title, recom.cosine_score)
    recommended_movies.show()




In [ ]:
# Get user input for movie title
title = 'Mortdecai'

print("\n")
print("Recommendation\n")
recommendation_by_title(title)